In [ ]:
import numpy as np # creating a short cut to acces numpy faster in future code
import matplotlib.pyplot as plt #importing matplotlib tool as a shortcut to be able to plot my conclusions. 
from numpy import loadtxt #importing a usefull tool from numpy

def read(file): #defining a new function to read in external files
    data = loadtxt(file) #calling the extracted file data
    data_new = data.transpose() # transposing the values, so I only got 3 rows of data and can acces it easier
    time = data_new[0] #the first row in this transposed data is the time the observations were made 
    flux = data_new[1] #the second row is the flux of the star
    error = data_new[2] #the last row is the error for the y-values(flux)
    
    return time, flux, error #returning three lists containing the unpacked values





files = ["transit_1.136-1.155micr.txt",
"transit_1.155-1.173micr.txt",
"transit_1.173-1.192micr.txt",

"transit_1.192-1.210micr.txt",

"transit_1.210-1.229micr.txt",

"transit_1.229-1.247micr.txt",

"transit_1.247-1.266micr.txt",

"transit_1.266-1.284micr.txt",

"transit_1.284-1.303micr.txt",

"transit_1.303-1.321micr.txt",

"transit_1.321-1.340micr.txt",

"transit_1.340-1.358micr.txt",

"transit_1.358-1.377micr.txt",

"transit_1.377-1.395micr.txt",

"transit_1.395-1.414micr.txt",

"transit_1.414-1.432micr.txt",

"transit_1.432-1.451micr.txt",

"transit_1.451-1.469micr.txt",

"transit_1.469-1.488micr.txt",

"transit_1.488-1.506micr.txt",

"transit_1.506-1.525micr.txt",

"transit_1.525-1.543micr.txt",

"transit_1.543-1.562micr.txt",

"transit_1.562-1.580micr.txt",

"wavEdgesMicr.txt"]
import batman  
from scipy.optimize import minimize
import math
import emcee



In [ ]:
time1, flux1, error1 = read(files[0]) #creating three large lists for all the time, flux and error data
nLCs = 24
time = np.full((nLCs,len(time1)), np.nan)

flux = np.full((nLCs,len(time1)), np.nan)
error = np.full((nLCs,len(time1)), np.nan)

for i in range(24):
    time1, flux1, error1 = read(files[i])
    
     
    time[i] = time1
    flux[i] = flux1
    error[i] = error1
flux2 = []

for i in range(24):
    for k in range (62):
        if 1 < flux[i][k]:
            flux2.append(flux[i][k])
      
flux = flux/(np.mean(flux2)) #creating a white light curve

time = np.concatenate(time)
flux = np.concatenate(flux)

error = np.concatenate(error)

In [ ]:
t = np.linspace(time[0], time[-1], len(time) )
import batman                         #importing the batman module to help plotting the transit curve

params = batman.TransitParams()       #object to store transit parameters
params.t0 =  2457424.88307                  #time of inferior conjunction
params.per = 1.2749255                       #orbital period
params.rp = 0.121                       #planet radius (in units of stellar radii)
params.a = 3.754                       #semi-major axis (in units of stellar radii)
params.inc = 89                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                        #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5, 0.1, 0.1, -0.1]      #limb darkening coefficients [u1, u2, u3, u4]


m = batman.TransitModel(params, t)    #initializes model
flux_bat = m.light_curve(params)          #calculates light curve
 
plt.plot(time, flux,"o")
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar
plt.plot(t, flux_bat)
plt.title(f"WASP-121 b", fontsize=40)
plt.xlabel('Time in Days', fontsize=20)
plt.ylabel('Flux ', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show() #hand fitting the parameter values

In [ ]:
def fun(x0,a , b, c): #creating a function to be minimised
   
    params.t0 = 2457424.88307                        #time of inferior conjunction
    params.per = 1.2749255                       #orbital period
    params.rp = x0[0]                       #planet radius (in units of stellar radii)
    params.a =3.754                       #semi-major axis (in units of stellar radii)
    params.inc = x0[1]                      #orbital inclination (in degrees)
    params.ecc = 0.                       #eccentricity
    params.w = 0.                       #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"        #limb darkening model
    params.u = [0.5,0.1,0.1,-0.1]
    
    
    
    
    flux_bat = m.light_curve(params)
    if  0.0 < x0[0] and  0.0 < x0[1] < 90  :
        return sum(((b-flux_bat)/c)**2)
    else:
         return float('inf')

In [ ]:
x0 = [0.121,  89 ] #first initial guesses for the values to be minimised
m = batman.TransitModel(params, time)
res = minimize(fun, x0, args = (time, flux, error), method='Nelder-Mead', tol=1e-6 )
x = res.x.tolist()

print(x)
theta = x

In [ ]:
x.insert(0,2457424.88307)# inserting the fixed values to the list fr future purposes
x.insert(1,1.2749255)
x.insert(3, 3.754)
true = np.asarray(x)


In [ ]:
m = batman.TransitModel(params, t)

params.t0 = x[0]                        #time of inferior conjunction
params.per = x[1]                     #orbital period
params.rp = x[2]                       #planet radius (in units of stellar radii)
params.a = x[3]                       #semi-major axis (in units of stellar radii)
params.inc = x[4]                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                       #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5,0.1,0.1,-0.1]
flux_bat1 = m.light_curve(params)

In [ ]:

fig = plt.figure(figsize=(10, 10)) # increasing the plot size to better distinguish between the values
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar
plt.plot(time, flux,"o") #plotting the time for the x-axis and the radial velocity for the y-axis
plt.plot(t, flux_bat1)
plt.title(f"WASP-121 b", fontsize=40)
plt.xlabel('Time in HJD', fontsize=20)
plt.ylabel('Flux ', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show() # plotting the light curve with the best fit values

In [ ]:
def log_likelihood(theta, time_fun, flux_fun, error_fun):# creating a log likelihood function for my emcee module
    
    
    params = batman.TransitParams()       #object to store transit parameters
    params.t0 = theta[0]                        #time of inferior conjunction
    params.per = theta[1]                 #orbital period
    params.rp = theta[2]                      #planet radius (in units of stellar radii)
    params.a = theta[3]                        #semi-major axis (in units of stellar radii)
    params.inc = theta[4]                      #orbital inclination (in degrees)
    params.ecc = 0.                       #eccentricity
    params.w = 0.                       #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"        #limb darkening model
    params.u = [0.5,0.1,0.1,-0.1]  
    model = m.light_curve(params)
    
    
   
    return -0.5 * (np.sum(((flux_fun - model) / error_fun)**2)+np.sum(((2457424.88307 - theta[0]) / 0.00011)**2)+ np.sum(((1.2749255-theta[1]) / 0.00000025)**2)+ np.sum(((3.754-theta[3]) / 0.028)**2))

In [ ]:
def log_prior(theta):#creating a log prior that checks if the values are in the physical boundaries
    
    if   0.0 < theta[2] < 1 and 0.0 < theta[3] < 50 and 0.0 < theta[4] < 90  :
        return 0.0 
  
        
    return -np.inf

In [ ]:
def log_probability(theta, time_fun, flux_fun, error_fun): #creating a log probability function
    lp = log_prior(theta)
    
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, time_fun, flux_fun, error_fun)

In [ ]:


m = batman.TransitModel(params, time)
ndim = 5
nwalkers = 50
pos = true + 1e-4 * np.random.randn(50, 5)
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=( time, flux, error), 
moves=emcee.moves.StretchMove())
sampler.run_mcmc(pos , 6000, progress=True);#letting my emcee run 6000 times

In [ ]:
labels = ["Time(0)","Orbital" "Period","Planet"" Radius", "Semi-Major"" Axis" ,"Orbital"" Inclination"]#creating a list of labels for my plot
fig, axes = plt.subplots(5, figsize=(10, 7), sharex=True)
samples = sampler.get_chain(discard = 1000)

for i in range(ndim): #creating five plots for the MCMC values
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])

In [ ]:
labels = ["Time" "" " of" " Inferior" "Conjunction","Orbital" " Period","Planet"" Radius", "Semi-Major"" Axis" ,"Orbital"" Inclination"]
import corner

flat_samples = sampler.get_chain( flat=True, discard = 1000) #creating corner plots for the results 
fig = corner.corner(
    flat_samples,   truths= true, labels = labels)

In [ ]:
from IPython.display import display, Math
results = []
result_uncertainty = []
for i in range(ndim): #printing the mean values and the corresponding one-sigma value
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.8f}}}^{{{2:.8f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))
    results.append(mcmc[1])
    result_uncertainty.append(q[1])

In [ ]:
print(result_uncertainty[3], result_uncertainty[4])

In [ ]:
%matplotlib notebook
m = batman.TransitModel(params, t)
params = batman.TransitParams()       #object to store transit parameters
params.t0 = results[0]                     #time of inferior conjunction
params.per = results[1]                      #orbital period
params.rp = results[2]                       #planet radius (in units of stellar radii)
params.a = results[3]                        #semi-major axis (in units of stellar radii)
params.inc = results[4]                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                       #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5,0.1,0.1,-0.1] 
    #initializes model
  


fig = plt.figure(figsize=(10, 10)) # increasing the plot size to better distinguish between the values
plt.plot(time, flux,"o")
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar
plt.plot(t, flux_bat)
plt.title(f"WASP-121 b", fontsize=40)
plt.xlabel('Time in Days', fontsize=20)
plt.ylabel('Flux ', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show() # plotting the data and the light curve with the parameter values from emcee